<a href="https://colab.research.google.com/github/CynYZhu/MIDS_266_FinalProject/blob/main/266_Project_Goding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W266 Project

In [1]:
#install tensorflow datasets library
!pip install datasets
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 6.0 MB/s 
     |████████████████████████████████| 140 kB 12.7 MB/s 
     |████████████████████████████████| 212 kB 40.2 MB/s 
     |████████████████████████████████| 101 kB 3.3 MB/s 
     |████████████████████████████████| 1.1 MB 30.5 MB/s 
     |████████████████████████████████| 596 kB 9.9 MB/s 
     |████████████████████████████████| 127 kB 39.4 MB/s 
     |████████████████████████████████| 144 kB 23.9 MB/s 
     |████████████████████████████████| 271 kB 19.2 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depen

In [2]:
from datasets import load_dataset
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from google.colab import files
import re

In [3]:
dataset_train, dataset_validation, dataset_test = load_dataset('trivia_qa', 'rc',split = ['train', 'validation', 'test'])

KeyboardInterrupt: ignored

In [ ]:
dataset_test[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
dataset_validation[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
dataset_train[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
#check how many empty search_context values there are to determine if 
counter = 0
for item in dataset_train:
  if item['search_results']['search_context'] != []:
    continue   
  else:
    counter +=1 

In [ ]:
print('There are ' + str(counter) + ' empty search_context in the dataset_train')

There are 65038 empty search_context in the dataset_train


In [ ]:
#total length of dataset_train
len(dataset_train)

138384

In [17]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/w266/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset_test  dataset_train  dataset_validation


In [ ]:
# #open first chunk
# pickle_off = open('drive/MyDrive/w266/file1.txt', 'rb')
# test_1 = pickle.load(pickle_off)

In [13]:
def clean_text(text):
  stop = stopwords.words('english')
  text = text.lower()
  text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
  return text

In [14]:
#function to format train, validation, and test data
def format_dataset(dataset):

  #extract needed information
  question_id = dataset['question_id']
  question = dataset['question']
  #clean up question text, context by lower case, removing unicode chars, removing stop words
  question = clean_text(question)
  #wiki_context = dataset['entity_pages']['wiki_context']
  search_context = dataset['search_results']['search_context']
  #evaluate each item in search_context list and make one large context
  total_context = []
  for item in search_context:
    item = clean_text(item)
    total_context.append(item) 

  #search_context = clean_text(search_context[0])
  #normalized aliases
  answer = dataset['answer']['normalized_aliases']

  return {'question_id': question_id,  'question': question, 'search_context': total_context, 'answer': answer}


In [15]:
cleaned_traindata =[]
for item in dataset_train:
  if item['search_results']['search_context'] != []:
    cleaned_traindata.append(format_dataset(item))
  else:
    continue

In [16]:
len(cleaned_traindata)

73346

In [ ]:
#save cleaned_traindata
# with open('cleaned_traindata.txt', 'wb') as f1:
#    pickle.dump(cleaned_traindata, f1)

In [ ]:
# #create new list with only search context items. Perform in batches because of limited RAM
#create first chunk
dataset_train_search = []
for item in cleaned_traindata:
  if item['search_results']['search_context'] != []:
    dataset_train_search.append(item)
  else:
    continue


 